# DAQHandler: High-Level Single-Task Interface

DAQHandler provides a unified interface for NI-DAQmx data acquisition and generation. It consolidates LDAQ's acquisition/generation patterns and OpenEOL's lifecycle management into a single class.

**Key features:**
- Accepts NI MAX task names (strings) or AITask/AOTask objects
- Software-triggered acquisition via pyTrigger
- Continuous signal generation
- Single-sample I/O for control applications
- Digital I/O support
- Auto-reconnection on device loss
- Thread-safe via RLock

## Environment Setup

Try to import nidaqmx. If unavailable, set up lightweight mocks so the examples run anywhere.

In [ ]:
import sys
from unittest.mock import MagicMock, Mock
import numpy as np

HW_AVAILABLE = False

try:
    import nidaqmx
    from nidaqmx import constants
    HW_AVAILABLE = True
    print("✓ Real NI-DAQmx available")
except ImportError:
    print("✗ nidaqmx not available — using mock mode")
    
    # Mock nidaqmx module
    nidaqmx = MagicMock()
    sys.modules['nidaqmx'] = nidaqmx
    
    # Mock constants
    constants = MagicMock()
    constants.AcquisitionType.CONTINUOUS = 10123
    constants.TerminalConfiguration.DEFAULT = 10083
    constants.VoltageUnits.VOLTS = 10348
    constants.AccelUnits.G = 10186
    constants.AccelSensitivityUnits.MILLIVOLTS_PER_G = 12509
    constants.READ_ALL_AVAILABLE = -1
    constants.RegenerationMode.ALLOW_REGENERATION = 10097
    nidaqmx.constants = constants
    sys.modules['nidaqmx.constants'] = constants
    
    # Mock System.local()
    mock_device = Mock()
    mock_device.name = "Dev1"
    mock_device.product_type = "PCIe-6320"
    
    mock_system = Mock()
    mock_system.devices = [mock_device]
    mock_system.tasks.task_names = ["MyInputTask", "MyOutputTask"]
    
    nidaqmx.system.System.local.return_value = mock_system
    
    # Mock Task
    def mock_task_factory(new_task_name=None):
        task = Mock()
        task.name = new_task_name or "mock_task"
        task.channel_names = ["Dev1/ai0", "Dev1/ai1"]
        task.number_of_channels = 2
        task.timing.samp_clk_rate = 25600.0
        task.devices = [mock_device]
        task.ai_channels = []
        task.ao_channels = []
        task.read.return_value = np.random.randn(2)
        task.is_task_done.return_value = False
        return task
    
    nidaqmx.task.Task = mock_task_factory
    
    # Mock get_task_by_name to return mock tasks
    def mock_get_task_by_name(name):
        if name in ["MyInputTask", "MyOutputTask"]:
            return mock_task_factory(name)
        return None
    
    # Mock get_connected_devices
    def mock_get_connected_devices():
        return {"Dev1"}

# Now import nidaqwrapper (works with real or mock nidaqmx)
from nidaqwrapper import DAQHandler, AITask, AOTask, DITask, DOTask

# Patch utils if in mock mode
if not HW_AVAILABLE:
    import nidaqwrapper.utils
    nidaqwrapper.utils.get_task_by_name = mock_get_task_by_name
    nidaqwrapper.utils.get_connected_devices = mock_get_connected_devices

print(f"Mode: {'Hardware' if HW_AVAILABLE else 'Mock'}")

## Basic Usage: NI MAX Task Names

The simplest way to use DAQHandler is with NI MAX task names. Create tasks in NI Measurement & Automation Explorer, then reference them by name.

In [ ]:
# Create handler with NI MAX task name
handler = DAQHandler(task_in='MyInputTask')

# Connect to hardware
connected = handler.connect()
print(f"Connected: {connected}")

if connected:
    # Get device info
    info = handler.get_device_info()
    print(f"Input channels: {info['input']['channel_names']}")
    print(f"Sample rate: {info['input']['sample_rate']} Hz")
    
    # Read a single sample from each channel
    data = handler.read()
    print(f"Single sample: {data}")
    print(f"Shape: {data.shape} — (n_channels,)")
    
    # Disconnect
    handler.disconnect()
    print("Disconnected")

## Programmatic Tasks: AITask/AOTask Objects

For programmatic control, create AITask/AOTask objects and pass them to DAQHandler.

In [ ]:
# Create AI task programmatically
ai_task = AITask("accel_test", sample_rate=25600)
ai_task.add_channel(
    channel_name="accel_x",
    device_ind=0,
    channel_ind=0,
    sensitivity=100.0,
    sensitivity_units="mV/g",
    units="g"
)
ai_task.add_channel(
    channel_name="accel_y",
    device_ind=0,
    channel_ind=1,
    sensitivity=100.0,
    sensitivity_units="mV/g",
    units="g"
)

# Create AO task programmatically
ao_task = AOTask("sig_gen", sample_rate=10000)
ao_task.add_channel("ao_0", device_ind=0, channel_ind=0)

# Configure handler with both tasks
handler = DAQHandler()
handler.configure(task_in=ai_task, task_out=ao_task)

# Connect
if handler.connect():
    print(f"Connected with {handler.get_channel_names()}")
    handler.disconnect()

## Reading Data

DAQHandler provides two read methods:
- `read()` — single sample per channel, returns (n_channels,)
- `read_all_available()` — drains buffer, returns (n_samples, n_channels)

In [ ]:
handler = DAQHandler(task_in='MyInputTask')

if handler.connect():
    # Single sample read
    sample = handler.read()
    print(f"Single sample: {sample}")
    print(f"Shape: {sample.shape}")
    
    # Read all available samples (drains buffer)
    # In mock mode this returns empty; on real hardware it drains the FIFO
    all_data = handler.read_all_available()
    print(f"\nAll available data shape: {all_data.shape}")
    print(f"Format: (n_samples, n_channels)")
    
    handler.disconnect()

## Writing Data: Single Sample

The `write()` method writes a single DC value to each output channel. Internally it handles nidaqmx's 2-sample minimum buffer requirement.

In [ ]:
# Create output-only handler
handler = DAQHandler(task_out='MyOutputTask')

if handler.connect():
    # Write single value (for single-channel tasks)
    handler.write(2.5)
    print("Wrote 2.5 V to output channel")
    
    # For multi-channel tasks, pass an array
    handler.write([1.0, -1.0])
    print("Wrote [1.0, -1.0] V to two channels")
    
    # Write zeros to all channels (safe shutdown)
    handler.write(np.zeros(2))
    print("Wrote zeros")
    
    handler.disconnect()

## Continuous Generation

The `generate()` method starts continuous signal generation. Data format is `(n_samples, n_channels)`. The hardware loops the waveform continuously.

In [ ]:
# Generate a 1 Hz sine wave
sample_rate = 10000  # Hz
duration = 1.0  # seconds
n_samples = int(sample_rate * duration)
t = np.linspace(0, duration, n_samples, endpoint=False)

# Single-channel sine wave
signal = 2.0 * np.sin(2 * np.pi * 1.0 * t)

handler = DAQHandler(task_out='MyOutputTask')

if handler.connect():
    # Start generation
    handler.generate(signal)
    print(f"Generating {signal.shape[0]} samples continuously")
    
    # Signal plays in a loop until stopped
    # In a real application, you'd wait or do other work here
    
    # Stop generation and write zeros
    handler.stop_generation()
    print("Generation stopped, outputs zeroed")
    
    handler.disconnect()

## Multi-Channel Generation

For multi-channel generation, data must be `(n_samples, n_channels)`.

In [ ]:
# Generate 1 Hz and 2 Hz sine waves on two channels
sample_rate = 10000
duration = 1.0
n_samples = int(sample_rate * duration)
t = np.linspace(0, duration, n_samples, endpoint=False)

# Two channels: different frequencies
ch1 = 1.0 * np.sin(2 * np.pi * 1.0 * t)
ch2 = 0.5 * np.sin(2 * np.pi * 2.0 * t)

# Stack into (n_samples, 2) format
signal = np.column_stack((ch1, ch2))
print(f"Signal shape: {signal.shape}")

# Create 2-channel output task
ao_task = AOTask("dual_gen", sample_rate=sample_rate)
ao_task.add_channel("ao0", device_ind=0, channel_ind=0)
ao_task.add_channel("ao1", device_ind=0, channel_ind=1)

handler = DAQHandler(task_out=ao_task)

if handler.connect():
    handler.generate(signal)
    print("Generating 1 Hz on ch0, 2 Hz on ch1")
    
    handler.stop_generation()
    handler.disconnect()

## Software-Triggered Acquisition

Use `set_trigger()` + `acquire()` for software-triggered data collection via pyTrigger. This is useful for capturing transient events.

In [ ]:
# Note: pyTrigger must be installed for this to work
# pip install git+https://github.com/your-org/pyTrigger.git

try:
    import pyTrigger
    
    handler = DAQHandler(task_in='MyInputTask')
    
    if handler.connect():
        # Configure trigger: 1000 samples after channel 0 crosses 0.5 V
        handler.set_trigger(
            n_samples=1000,
            trigger_channel=0,
            trigger_level=0.5,
            trigger_type='up',  # 'up', 'down', or 'abs'
            presamples=100  # Capture 100 samples before trigger
        )
        
        # Blocking acquisition
        print("Waiting for trigger...")
        data = handler.acquire(blocking=True)
        print(f"Acquired shape: {data.shape}")
        
        # Or return as dict with channel names
        data_dict = handler.acquire(return_dict=True, blocking=True)
        print(f"Dict keys: {list(data_dict.keys())}")
        
        handler.disconnect()

except ImportError:
    print("pyTrigger not installed — skipping triggered acquisition example")

## Acquisition Loop Pattern

For continuous monitoring, use `read_all_available()` in a loop.

In [ ]:
import time

handler = DAQHandler(task_in='MyInputTask')

if handler.connect():
    print("Starting acquisition loop (5 iterations)...")
    
    for i in range(5):
        data = handler.read_all_available()
        
        if data.shape[0] > 0:
            rms = np.sqrt(np.mean(data**2, axis=0))
            print(f"Iteration {i}: {data.shape[0]} samples, RMS: {rms}")
        else:
            print(f"Iteration {i}: No data available")
        
        time.sleep(0.1)
    
    handler.disconnect()
    print("Acquisition stopped")

## Digital I/O

DAQHandler supports digital input/output via DITask/DOTask objects.

In [ ]:
# Create digital tasks
di_task = DITask("digital_read")
di_task.add_channel('switch_inputs', lines='Dev1/port0/line0:3')  # 4 lines

do_task = DOTask("digital_write")
do_task.add_channel('led_outputs', lines='Dev1/port0/line4:7')  # 4 lines

# Configure handler with digital tasks
handler = DAQHandler()
handler.configure(task_digital_in=di_task, task_digital_out=do_task)

if handler.connect():
    # Read digital inputs
    input_state = handler.read_digital()
    print(f"Digital inputs: {input_state}")
    
    # Write digital outputs
    handler.write_digital([True, False, True, False])
    print("Wrote digital pattern: [True, False, True, False]")
    
    # Set all outputs low
    handler.write_digital([False, False, False, False])
    print("All digital outputs set to False")
    
    handler.disconnect()

In [ ]:
handler = DAQHandler(task_in='MyInputTask')

if handler.connect():
    # Simulate work that doesn't read data
    time.sleep(0.5)
    
    # Buffer now contains ~0.5 seconds of data
    # Discard it
    handler.clear_buffer()
    print("Buffer cleared")
    
    # Next read gets fresh data
    data = handler.read_all_available()
    print(f"Fresh data: {data.shape}")
    
    handler.disconnect()

## Connectivity: ping() and check_state()

Check device connectivity and recover from disconnections.

In [ ]:
handler = DAQHandler(task_in='MyInputTask')

if handler.connect():
    # ping() checks if all required devices are connected
    is_connected = handler.ping()
    print(f"Ping result: {is_connected}")
    
    # check_state() attempts auto-reconnection if needed
    state = handler.check_state()
    print(f"State: {state}")
    # Possible states: 'connected', 'reconnected', 'connection lost', 'disconnected'
    
    handler.disconnect()

## Introspection: Device Info

Query channel names, sample rates, and device metadata.

In [ ]:
handler = DAQHandler(task_in='MyInputTask', task_out='MyOutputTask')

if handler.connect():
    # Get full device info
    info = handler.get_device_info()
    print("Device info:")
    for key, val in info.items():
        print(f"  {key}: {val}")
    
    # Get specific properties
    channels = handler.get_channel_names()
    sample_rate = handler.get_sample_rate()
    
    print(f"\nInput channels: {channels}")
    print(f"Sample rate: {sample_rate} Hz")
    
    handler.disconnect()

## Context Manager

Use DAQHandler as a context manager for automatic cleanup.

In [ ]:
# Context manager ensures disconnect() is called
with DAQHandler(task_in='MyInputTask') as handler:
    if handler.connect():
        data = handler.read()
        print(f"Read: {data}")
# Automatically disconnected here
print("Handler automatically disconnected")

## Reconfiguration

Call `configure()` to change tasks without creating a new handler instance.

In [ ]:
handler = DAQHandler()

# Initial configuration
handler.configure(task_in='MyInputTask')
if handler.connect():
    data = handler.read()
    print(f"First config read: {data}")
    handler.disconnect()

# Reconfigure with different task
handler.configure(task_out='MyOutputTask')
if handler.connect():
    handler.write(1.5)
    print("Second config wrote 1.5 V")
    handler.disconnect()

print("Reconfiguration allows reusing the same handler instance")

## Timing Parameters

Control acquisition loop timing via `acquisition_sleep` and `post_trigger_delay`.

In [ ]:
# Configure timing parameters
handler = DAQHandler(
    task_in='MyInputTask',
    acquisition_sleep=0.05,  # Sleep 50 ms between buffer reads
    post_trigger_delay=0.1   # Wait 100 ms after trigger completes
)

# Or set via configure()
handler.configure(
    task_in='MyInputTask',
    acquisition_sleep=0.02,
    post_trigger_delay=0.05
)

print(f"Acquisition sleep: {handler.acquisition_sleep} s")
print(f"Post-trigger delay: {handler.post_trigger_delay} s")

## Summary

**DAQHandler provides:**
- Unified interface for NI MAX tasks and programmatic tasks
- Single-sample I/O: `read()`, `write()`
- Buffered I/O: `read_all_available()`, `generate()`
- Software triggering: `set_trigger()`, `acquire()`
- Digital I/O: `read_digital()`, `write_digital()`
- Device management: `connect()`, `disconnect()`, `ping()`, `check_state()`
- Buffer control: `clear_buffer()`, `stop_generation()`
- Context manager support

**Data format:**
- Single sample: `(n_channels,)`
- Multiple samples: `(n_samples, n_channels)`

**Thread-safe:** All methods use a per-instance RLock.

For multi-task applications (synchronized input/output, hardware triggering), use MultiHandler instead.